# Modificación del dataset de películas

Importación de módulos a utilizar:

In [22]:
import os
import csv
import itertools
import string
from collections import Counter
from functools import reduce
import random

Funciones definidas:

In [23]:
def elijo(n,secuencia):
    """Retorna una lista de n items elegidos al azar de una secuencia"""
    return random.sample(secuencia,n)

Se abre archivo con el que se va a trabajar:

In [24]:
ruta=os.getcwd()
arch = 'mymoviedb.csv'
arch_nuevo = 'data_set_peliculas.csv'
try:
    with open(os.path.join(ruta,'archivos',arch),'r',encoding='utf-8') as archivo:
        reader = csv.reader(archivo,delimiter=',')
        
        #PRIMERO SE DEBE OBTENER LAS 100 PALABRAS QUE MÁS APARECEN EN LOS OVERVIEWS
        
        #Además, hay que descartar las películas que no tienen “overview” y las películas cuyo idioma original tenga más de 2 caracteres.
        next(reader)
        
        descarte = filter(lambda x: x[2]!='' and len(x[6])<3,reader)
        

        #Se separan las palabras de la columna del overview. Quedaría una lista de palabras por línea
        overview = map(lambda x: x[2].split(' '),descarte)
 
        #Se agrupan las palabras de todos los overview en una sola lista
        palabras_overview = list(itertools.chain.from_iterable(overview))

        #Se toman las 100 palabras más comunes de todos los overviews combinados.
        top_100 = Counter(palabras_overview).most_common(100)
        
        #Se obtiene lista de tuplas, donde el primer elemento es la palabra y el segundo las veces que aparece. 
        #Se obtiene una lista con los elementos pertenecientes a la primera posición de cada tupla.
        top_100 = [x[0] for x in top_100]
        
        #SE OBTUVO LA LISTA TOP_100 DE PALABRAS DE LOS OVERVIEW
        
        #Creo el nuevo archivo
        try:
            with open(os.path.join(ruta,'archivos',arch_nuevo),'x',encoding='utf-8') as nuevo:
                #Comienzo del archivo original
                archivo.seek(0)
                
                #Se ecribe encabezado en el nuevo archivo
                a = next(reader)
                lista_encabezado = [a[7],a[6],a[0],a[5],a[2],a[1]]
                nuevo.write(','.join(lista_encabezado) + '\n')
                
                
                #EN EL NUEVO ARCHIVO, LA COLUMNA DEL OVERVIEW NO DEBE POSEER LAS PALABRAS PERTENECIENTES AL TOP_100
                #ADEMÁS DEBE TENER HASTA 3 PALABRAS.   
                              
                while True:
                    try:
                        
                        linea = next(descarte)
                        
                        #Se obtiene lista de palabras del overview
                        palabras_overview = linea[2].split()
                        
                        #Se eliminan del overview las palabras pertenecientes al top 100
                        palabras_sin_top_100 = [x for x in palabras_overview if x.strip(string.punctuation) not in top_100]
                        
                        try:
                            #Se elije en forma aleatoria 3 palabras para cada overview
                            columna_overview = elijo(3,palabras_sin_top_100)
                        
                        #Si el overview filtrado posee menos de 3 será una excepción y se deberán elegir menos.
                        except ValueError:
                            try:
                                columna_overview = elijo(2,palabras_sin_top_100)
                       
                            except ValueError:
                                columna_overview = elijo(1,palabras_sin_top_100)
                        finally:
                            
                            overview = ' '.join(columna_overview)
                            
                            columnas = ",".join([linea[7],linea[6],linea[0],linea[5],overview,linea[1]])

                            nuevo.write(columnas + '\n')
                    
                    except StopIteration:
                        break          
        
        except FileExistsError:
            print(f"El archivo {arch_nuevo} ya existe")

except FileNotFoundError:
    print(f"No se encuentra el archivo {arch}")
